In [ ]:
!pip install requests python-dotenv beautifulsoup4 openai

In [ ]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

import os
from openai import OpenAI
from dotenv import load_dotenv

# Set environment variable directly (alternative to .env file)
os.environ['OPENAI_API_KEY'] = "sk........"

# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Sanity check for API key
if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

# Model name and OpenAI client initialization
MODEL = 'gpt-4o-mini'
openai = OpenAI()


API key looks good so far


In [ ]:
class TextFile:
    """
    A utility class to represent a local text file, mimicking the Website class structure.
    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.title = file_path.split("/")[-1]  # Use filename as title

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                self.text = f.read().strip()
        except FileNotFoundError:
            print(f"[ERROR] File not found: {file_path}")
            self.text = ""
        except Exception as e:
            print(f"[ERROR] Failed to read file: {e}")
            self.text = ""

    def get_contents(self):
        return f"File Title:\n{self.title}\n\nFile Contents:\n{self.text}\n\n"


In [ ]:
file = TextFile("rxsme.txt")
print(file.get_contents())

File Title:
rxsme.txt

File Contents:
==RESUME 1==

Priya Sharma  
Email: priya.sharma@gmail.com | Phone: +91-9876543210  
LinkedIn: linkedin.com/in/priyasharma  

Objective  
Detail-oriented data enthusiast with hands-on experience in analytics and dashboard creation, seeking roles in AI and analytics.

Education  
B.Tech in Electronics and Communication Engineering  
Delhi Technological University | 2019 – 2023

Skills  
- Python, SQL, Excel  
- Data Visualization (Tableau)  
- Statistical Analysis

Experience  
Data Analyst Intern | Infosys | Jan 2023 – June 2023  
- Built customer churn prediction dashboards in Tableau  
- Cleaned and analyzed large telecom datasets using Python

Certifications  
- Google Data Analytics Certificate  

--------------------------------------------------------------------------------------------------------------------------

==RESUME 2==

Rohit Mehta  
Email: rohit.mehta@gmail.com | Phone: +91-9012345678  
GitHub: github.com/rohitmehta | LinkedIn: li

In [ ]:
system_prompt = (
    """
You are an AI assistant helping a hiring manager evaluate multiple job resumes for the position of A.I. Engineer.

Job Requirements:
- A college degree (any discipline)
- Proficiency in programming/coding
- Background in data analytics
- Strong skills in Artificial Intelligence (A.I.) and Machine Learning (M.L.)

Instructions:
1. For each resume, assign a score from 0–100 based on how well it meets the job requirements.
2. Rank the resumes from most suitable to least.
3. Justify each score with a brief explanation.
4. If a resume is not suitable, state what the candidate lacks.

The resumes are marked clearly as RESUME 1, RESUME 2, etc.
"""
)


In [ ]:
def get_user_prompt(file):
    user_prompt = "The following are the resumes of candidates who have applied for a job for a role of an A.I Engineer. \n"
    user_prompt += "you are required to assign suitable scores, rank them accordingly and give a brief explaination for each."
    user_prompt += "Also mention if the given resume is suitable for the job or not."
    user_prompt += file.get_contents()
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [ ]:
get_user_prompt(file)

'The following are the resumes of candidates who have applied for a job for a role of an A.I Engineer. \nyou are required to assign suitable scores, rank them accordingly and give a brief explaination for each.Also mention if the given resume is suitable for the job or not.File Title:\nrxsme.txt\n\nFile Contents:\n==RESUME 1==\n\nPriya Sharma  \nEmail: priya.sharma@gmail.com | Phone: +91-9876543210  \nLinkedIn: linkedin.com/in/priyasharma  \n\nObjective  \nDetail-oriented data enthusiast with hands-on experience in analytics and dashboard creation, seeking roles in AI and analytics.\n\nEducation  \nB.Tech in Electronics and Communication Engineering  \nDelhi Technological University | 2019 – 2023\n\nSkills  \n- Python, SQL, Excel  \n- Data Visualization (Tableau)  \n- Statistical Analysis\n\nExperience  \nData Analyst Intern | Infosys | Jan 2023 – June 2023  \n- Built customer churn prediction dashboards in Tableau  \n- Cleaned and analyzed large telecom datasets using Python\n\nCertif

In [ ]:
def stream_brochure(file):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(file)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure(file)

**Score and Ranking of Resumes:**

1. **Rohit Mehta**  
   **Score: 95**  
   **Justification:** Rohit has a strong background in Artificial Intelligence with a degree in Computer Science and hands-on experience in machine learning and deep learning. His skills include Python, TensorFlow, and PyTorch, which are highly relevant for A.I. engineering. He has worked on significant projects like sentiment analysis and fake news detection, showcasing his proficiency and interest in AI applications. His resume meets all job requirements effectively.  

2. **Kunal Raj**  
   **Score: 90**  
   **Justification:** Kunal possesses an M.Sc. in Data Science, alongside a B.Sc. in Statistics, which provides him a solid foundation in analytics. His skills in Python, R, SQL, and machine learning methods (including XGBoost) make him a strong candidate. His experience with customer segmentation and sales prediction demonstrates relevant practical application in data-driven environments. He is highly suitable due to his education and experience in analytics and AI.

3. **Priya Sharma**  
   **Score: 75**  
   **Justification:** Priya holds a B.Tech in Electronics and Communication Engineering and has relevant skills in Python and data visualization. However, while she has experience as a data analyst intern, her focus has been more centered on data analytics rather than on AI and machine learning specifically. She lacks experience with advanced AI/Machine Learning techniques and frameworks, which limits her suitability for the role compared to the previous candidates.

4. **Aarti Desai**  
   **Score: 20**  
   **Justification:** Aarti has a background in sociology, which does not provide the technical foundation required for an A.I. Engineer. Her skills are not aligned with programming or data analytics, and her experience is not related to AI or machine learning. She lacks education, skills, and experience suitable for the job requirements, making her an unsuitable candidate. 

**Summary of Suitability:**

- **Rohit Mehta:** Highly suitable; strong AI/ML background and relevant experience.  
- **Kunal Raj:** Suitable; solid educational background and experience in data science with relevant skills.  
- **Priya Sharma:** Moderately suitable; has a data analytics background but lacks advanced AI/ML expertise.  
- **Aarti Desai:** Not suitable; lacks the required background and skills for the A.I. Engineer position.

In [ ]:
file2 = TextFile("rxsume2.txt")
print(file2.get_contents())

File Title:
rxsume2.txt

File Contents:
Name: John Doe
Email: johndoe@example.com
Phone: +1 (123) 456-7890
LinkedIn: linkedin.com/in/johndoe
GitHub: github.com/johndoe

Objective
Aspiring A.I. Engineer with a strong background in computer science, machine learning, and data analytics. Seeking to apply advanced AI techniques to real-world problems and contribute to impactful innovations in artificial intelligence.

Education
Bachelor of Technology in Computer Science
Indian Institute of Technology, Delhi — 2019–2023

GPA: 8.6/10

Relevant Coursework: Machine Learning, Deep Learning, Data Mining, Algorithms

Skills
Programming Languages: Python, C++, Java

AI/ML Libraries: TensorFlow, PyTorch, scikit-learn, Keras

Data Analytics: Pandas, NumPy, Matplotlib, Power BI

Other Tools: Git, Docker, Jupyter, SQL, REST APIs

Soft Skills: Problem-solving, teamwork, analytical thinking

Projects
Fake News Detection using NLP and Deep Learning

Built an LSTM-based neural network using Keras to detec

In [ ]:
stream_brochure(file2)

**Score: 95/100**

**Ranking: 1st**

**Justification:**
John Doe's resume meets all job requirements exceptionally well. He has a relevant college degree in Computer Science from a prestigious institution and demonstrates proficiency in programming with languages including Python, C++, and Java. His coursework in Machine Learning, Deep Learning, and Data Mining aligns perfectly with the job's focus. His experience with AI/ML libraries like TensorFlow and PyTorch showcases his strong skills in artificial intelligence and machine learning. Additionally, his work experience as a Machine Learning Intern and practical projects such as Fake News Detection and Autonomous Driving Simulation further validate his expertise in the field. Moreover, the inclusion of certifications enhances his profile, adding credibility to his skills. The only minor drawback is a lack of direct mention of data analytics role in his projects, but the skills listed suggest adequate proficiency in this area.
  
---

Next, please provide the details of the next resume (RESUME 2) for evaluation.